# Testing our LRU Cache for Arbitrage Optimization

The biggest bottleneck in our code is finding the optimal arbitrage for our trades. One way we have optimized this is by implementing an LRU cache on the `cycle.optimize()` method. Here, we test that it works.

In [1]:
# TODO move this to a test file
import time
from src.sim.scenario import Scenario
from src.trades.cycle import _optimize_mem

In [2]:
# Generate markets and prices
scenario = Scenario("baseline", "wsteth")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()  # Set block timestamps
scenario.prepare_for_trades(sample)  # Set External Market Price

[INFO][18:48:45][src.sim.scenario]-16454: Fetching sim_market from subgraph.
[INFO][18:48:55][src.utils.poolgraph]-16454: Found 20 valid trading cycles of length 3.


In [3]:
# Unpack
cycles = scenario.cycles
cycle = cycles[0]
trade = cycle.trades[0]
decimals = trade.pool.coin_decimals[trade.i]
address = trade.pool.coin_addresses[trade.i].lower()

# Set xatol
xatol = int(10**decimals / sample.prices_usd[address])

In [4]:
# Check single cycle caching
start = time.time()
cycle.optimize(xatol)
end = time.time()
print(f"Time before caching: {end - start}")
start = time.time()
amt, opt_profit = cycle.optimize(xatol)
end = time.time()
print(f"Time after caching: {end - start}")
_optimize_mem.cache_info()  # Ensure there was one hit and one miss!

Time before caching: 0.0007369518280029297
Time after caching: 3.218650817871094e-05


CacheInfo(hits=1, misses=1, maxsize=100, currsize=1)

In [5]:
pop_profit = cycle.populate(amt, use_snapshot_context=False)
exc_profit = cycle.execute()

assert (
    pop_profit == exc_profit == opt_profit
)  # also check the functions are consistent!

In [6]:
amt, opt_profit = cycle.optimize(xatol)
_optimize_mem.cache_info()  # Ensure the cache wasn't hit this time

CacheInfo(hits=1, misses=2, maxsize=100, currsize=2)

In [7]:
cycle.optimize(xatol)
_optimize_mem.cache_info()  # Ensure the cache was hit this time

CacheInfo(hits=2, misses=2, maxsize=100, currsize=2)

In [8]:
sample = scenario.pricepaths[1]
xatol = int(10**decimals / sample.prices_usd[address])
scenario.prepare_for_trades(sample)  # Update External Market Price
cycle.optimize(xatol)
_optimize_mem.cache_info()  # Ensure the cache wasn't hit this time because prices changed

CacheInfo(hits=2, misses=3, maxsize=100, currsize=3)

In [9]:
# Nice!